In [1]:
import os
import cv2
import math
import json
import glob
import random
import numpy as np
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch
import torch.nn.functional as F

from utils.sdf import SDF
from utils.util import load_mask, load_png, load_sdm, logits_to_lbl
from utils.dataset import DatasetTools
from configs.cfgparser import Config

from model.MobaNet import MobaNet

cfg  = Config('configs/config.yaml', inference = True, cli = False)
cfg.RANK = 0

from predict import Predictor

[INFO] Configuration file passed all validation tests.


In [ ]:
# imID = 'ESP_018244_2655_RED-3456_0'
# impath = f'{cfg.IMG_DIR}/{imID}.png'
# model = Predictor(cfg)
# output = model.predict(impath)

tensor([2], device='cuda:0')


In [70]:
# MASK STATES
# 1. UNet input:         1hot encoded; (B, C, H, W)
# 2. SDM generation:     1hot encoded; (B, C, H, W)
# 3. Losses DICE-based:  1hot encoded; (B, C, H, W)
# 4. Loss sdm-based:     1hot encoded; (B, C, H, W)
# 5. Losses CE-based:    Argmax();     (B, H, W)
# 6. Metrics:            1hot encoded; (B, C, H, W)

# SEG LOGIT STATES
# 1. Losses DICE-based:  Probabilistic [0; 1];     (B, C, H, W)
# 2. Loss sdm-based:     Tanh() sdm-union [-1; 1]; (B, 1, H, W)
# 3. Losses CE-based:    Direct logits;            (B, C, H, W)
# 4. Metrics:            1hot encoded;             (B, C, H, W)
# 5. Visualization:      Argmax()                  (B, 1, H, W)

# CLS LOGIT STATES
# 1. Loss CE:            Direct logits; (B, C)
# 2. Metrics:            Argmax()       (B, ) 
# 3. UNet forward():     Argmax()       (B, )

In [ ]:
# impath = cfg.IMG_DIR / 'ESP_018244_2655_RED-3456_0.png'
# img = load_png(impath)
# # img = img[None, ...]  # add batch dimension
# # tensor = torch.from_numpy(img).permute(0, 3, 1, 2).float() # (B, H, W, C) -> (B, C, H, W)
# tensor = torch.from_numpy(img).permute(2, 0, 1).float()  # (H, W, C) -> (C, H, W)


In [ ]:
# Post-processing for a clean line: After prediction, compute the boundary as the set of pixels where a 4- or 8-connected 
# neighborhood changes class, or subtract eroded from dilated masks to get a 1-pixel ridge.

(2, 512, 512)